In [1]:
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
from klepto.archives import dir_archive
import cv2
import time
from scipy.io import savemat

In [2]:
BEDFORMS = ['corals', 'canopy', 'rocks', 'dunes']
flow_speeds = ['fast', 'med', 'slow']
submergences = ['Deep', 'Intermed', 'Shallow']
colors = {'corals':'dodgerblue','canopy':'green', 'rocks':'orange', 'dunes':'brown'}

In [3]:
bed_idx = 0
flow_idx = 2
submergence_idx = 1
TEST = 2
BATCH_FRAMES =  75# for data acquired at 3 fp, total of 900, per subfolder 225 frames. 225/75 = 9
img_write_counter = 0
# SET the data variables
FLOW_SPEED = flow_speeds[flow_idx]
SUBMERGENCE = submergences[submergence_idx]
BEDFORM = BEDFORMS[bed_idx]

make folders to write the data to

In [4]:
# for FLOW_SPEED in flow_speeds:
#     for SUBMERGENCE in submergences:
#         d = '../GAN_trial_1_dunes/data/flat2sign/{}_{}Flow_{}Depth_{}'.format(BEDFORM, FLOW_SPEED, SUBMERGENCE, 'sx')
#         if not os.path.exists(d):
#             os.mkdir(d)

## PSD using 2D FFT

In [5]:
bed_data ={}
for serial, i in enumerate([1]):#,2,3,4]:#range(1,5):
    # choose the data
    SUBFOLDER = i

    # source data folders
    data_foldername = "{}_{}Flow_{}H_test{}_subFolder{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST, SUBFOLDER)
    print(data_foldername)
    data_location =  "../TOSHIBA_Drive/saksham_polar_cam_FLIR"
    save_location =  "../FLIR_Camera"
    klepto_saved_data_origin = 'klepto_bulk_data_saves' 

    # retrive the saved data
    data_origin = os.path.join(data_location, klepto_saved_data_origin, data_foldername)
    tic = time.time()
    assert os.path.exists(data_origin) == True
    data = dir_archive(data_origin, {}, serialized=True, cached=False)
#     bed_data['all_slopeMag_maps'], bed_data['all_slopeAzi_maps']\
#                 = cv2.cartToPolar(data['all_sy_maps'], data['all_sx_maps'], angleInDegrees=True)
#     bed_data['all_sy_maps'], bed_data['all_sx_maps'] = data['all_sy_maps'], data['all_sx_maps']
    bed_data['all_sx_maps'] = data['all_sx_maps']  
    del data
    
    toc = time.time()
    print("time taken to load the data  = ", toc-tic)

corals_slowFlow_IntermedH_test2_subFolder1
time taken to load the data  =  19.11237668991089


In [6]:
dst = '../GAN_trial_1_dunes/data/flat2sign/{}_{}Flow_{}Depth_{}'.format(BEDFORM, FLOW_SPEED, SUBMERGENCE, 'sx')
assert os.path.exists(dst)

In [7]:
dpi = 100
NUM_FRAMES = bed_data['all_sx_maps'].shape[2]
print(NUM_FRAMES)
for i in tqdm(range(NUM_FRAMES)):
    im_data  = bed_data['all_sx_maps'][:,:,i]

    height, width = im_data.shape

    # What size does the figure need to be in inches to fit the image?
    figsize = width / 2/float(dpi), height / 2/float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figsize)
    ax = fig.add_axes([0, 0, 1, 1])

    # Hide spines, ticks, etc.
    plt.axis('off')

    plt.imshow(im_data, cmap = plt.cm.inferno, vmin = -0.2, vmax = 0.1, interpolation='nearest')

    # Ensure we're displaying with square pixels and the right extent.
    # This is optional if you haven't called `plot` or anything else that might
    # change the limits/aspect.  We don't need this step in this case.
    ax.set(xlim=[-0.5, width - 0.5], ylim=[height - 0.5, -0.5], aspect=1)
    save_filename = '{}_{}Flow_{}Depth_{}_{}.jpg'.format(BEDFORM, FLOW_SPEED, SUBMERGENCE, 'sx', \
                                                         str(img_write_counter).zfill(5))
    fig.savefig(os.path.join(dst, save_filename), dpi=dpi)
    # plt.show()
    img_write_counter +=1
    plt.close()

225


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [00:30<00:00,  7.44it/s]


In [8]:
# plt.imshow(bed_data['all_slopeMag_maps'][:,:,20], cmap = plt.cm.inferno, interpolation='nearest')#, vmin = -0.2, vmax = 0.1)
# plt.colorbar()
# plt.title('slope mag')
# plt.axis('off')

# plt.figure()
# plt.imshow(bed_data['all_slopeAzi_maps'][:,:,20], cmap = plt.cm.inferno, interpolation='nearest')#, vmin = -0.2, vmax = 0.1)
# plt.colorbar()
# plt.title('slope azimuth')
# plt.axis('off')

# plt.figure()
# plt.imshow(bed_data['all_sx_maps'][:,:,20], cmap = plt.cm.inferno, vmin = -0.2, vmax = 0.1, interpolation='nearest')
# plt.colorbar()
# plt.title('Sx')
# plt.axis('off')

# plt.figure()
# plt.imshow(bed_data['all_sy_maps'][:,:,20], cmap = plt.cm.inferno, vmin = -0.2, vmax = 0.1, interpolation='nearest')
# plt.colorbar()
# plt.title('Sy')
# plt.axis('off')
